## Configuration

In [ ]:
# Generic imports and configuration.
%matplotlib inline
from matplotlib.pyplot import imshow, subplots
from numpy import array, zeros, ones
from colorsys import hsv_to_rgb
from random import randint

# Import Pybricks Colors.
from pybricks.parameters import Color

## Detectable colors

In [ ]:
# Load the default color map. We still need to revise BLACK to set a nonzero value to distinguish from None.
Color.BLACK = Color(0, 0, 20, 'BLACK')
color_map = (Color.RED, Color.BLUE, Color.YELLOW, Color.GREEN, Color.BLACK, Color.WHITE, None)

## Color map implementation

In [ ]:
# Integer implementation of Python's builtin hsv_to_rgb.
def pybricks_hsv_to_rgb(h, s, v):
    return [int(c*255) for c in hsv_to_rgb(h/360, s/100, v/100)]

# Highest possible error cost between measured and discrete color.
INT32_MAX = 2147483547

# The currently implemented HSV cost function. This is a simplistic implementation
# to keep the new code working like the old color detection algorithm. The purpose
# of this Jupyter Notebook is to improve this particular function and review the
# result below.
def pybricks_hsv_cost(x, c):
    if x.s < 40 or x.v <= 5:
        if c.s == 100:
            return INT32_MAX
        return x.v - c.v if x.v > c.v else c.v - x.v

    hue_error = c.h - x.h if c.h > x.h else x.h - c.h;
    return hue_error if hue_error <= 180 else 360 - hue_error

# Given a measured color, loop through all colors in color_map to find the best match.
def pybricks_discretize_color(measured):

    # Initially there is no match and the errors are at a maximum.
    match = None
    cost_now = INT32_MAX
    cost_min = INT32_MAX

    # Iterate through candidate colors.
    for compare in color_map:
        
        # For None, give values for comparison with measurement.
        if compare is None:
            compare = Color(0, 0, 0, 'None')

        # Evaluate the cost function for this candidate
        cost_now = pybricks_hsv_cost(measured, compare)

        # Update the minimum detected so far, and the corresponding matching color.
        if cost_now < cost_min:
            cost_min = cost_now
            match = compare

    # If None was the best match, return corresponding object.
    if match is not None and match.name == 'None':
        match = None
            
    return match

## Visualize

In [ ]:
# We make several HUE-VALUE plots for several steps of saturation.
SATURATION_INCREMENT = 25

# Define hue, saturation and value ranges.
saturation_range = range(100, -SATURATION_INCREMENT, -SATURATION_INCREMENT)
hue_range = range(360)
value_range = range(0, 101)

# Create the figure and its axes.
figure, axes = subplots(nrows=len(saturation_range), ncols=2, figsize=(12, 6*len(saturation_range)))

# Create empty matrices to hold measured and discretized colors.
image_measured = zeros((101, 360, 3), dtype=int)
image_discrete = zeros((101, 360, 4), dtype=int)

# Create a graph with a few grey lines so that we can see 0-alpha of the overlay.
image_alphadot = ones((101, 360, 3), dtype=int)*255
for i in hue_range:
    if (i//10) % 2:
        for j in value_range:
            image_alphadot[j, i] = (180, 180, 180)

# Loop through all discrete saturation steps that we will visualize.
for saturation_index, saturation in enumerate(saturation_range):

    # Loop through values 0--100.
    for value in value_range:
        # Loop through hues 0--359.
        for hue in hue_range:
            
            # Convert h, s, v for this iteration to RGB for visalization.
            image_measured[value][hue][:] = pybricks_hsv_to_rgb(hue, saturation, value)

            # Get the discrete color for this h, s, v.
            discrete = pybricks_discretize_color(Color(hue, saturation, value))
            
            # Display the discrete color.
            if discrete is None:
                # If it's None, don't show anything.
                image_discrete[value][hue][:] = 0, 0, 0, 0
            else:
                # Convert discrete h, s, v, to RGB or visualization.
                r, g, b = pybricks_hsv_to_rgb(discrete.h, discrete.s, discrete.v)
                image_discrete[value][hue][:] = r, g, b, 255

    # Plot the measured and the discrete colors for the current saturation step.
    axes[saturation_index][0].imshow(image_measured, aspect=3.6, origin='lower')
    axes[saturation_index][1].imshow(image_alphadot, aspect=3.6, origin='lower')
    axes[saturation_index][1].imshow(image_discrete, aspect=3.6, origin='lower')